# Import Required Libraries
Import the necessary libraries, including TensorFlow, Keras, and other dependencies.

In [1]:
# Import Required Libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, LeakyReLU, BatchNormalization, Dropout, Concatenate, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

# Load and Preprocess Dataset
Load the dataset and preprocess the images for training, including resizing and normalization.

In [ ]:
# Load and Preprocess Dataset

# Import additional required libraries
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Define function to load and preprocess images
def load_and_preprocess_image(image_path, target_size=(256, 256)):
    # Load image
    image = load_img(image_path, target_size=target_size)
    # Convert image to array
    image = img_to_array(image)
    # Normalize image to range [-1, 1]
    image = (image / 127.5) - 1.0
    return image

# Define function to load dataset
def load_dataset(dataset_path, target_size=(256, 256)):
    images = []
    for image_name in os.listdir(dataset_path):
        image_path = os.path.join(dataset_path, image_name)
        image = load_and_preprocess_image(image_path, target_size)
        images.append(image)
    return np.array(images)

# Example usage
train_images = load_dataset('data/mini_pix2pix/train')

# Print the shape of the datasets
print(f'Training set shape: {train_images.shape}')

# Define the Generator Model
Define the architecture of the generator model using Keras.

In [ ]:
# Define the Generator Model
def define_generator(image_shape):
    # Input layer
    inputs = Input(shape=image_shape)

    # Encoder
    down1 = Conv2D(64, (4, 4), strides=(2, 2), padding='same')(inputs)
    down1 = LeakyReLU(alpha=0.2)(down1)

    down2 = Conv2D(128, (4, 4), strides=(2, 2), padding='same')(down1)
    down2 = BatchNormalization()(down2)
    down2 = LeakyReLU(alpha=0.2)(down2)

    down3 = Conv2D(256, (4, 4), strides=(2, 2), padding='same')(down2)
    down3 = BatchNormalization()(down3)
    down3 = LeakyReLU(alpha=0.2)(down3)

    down4 = Conv2D(512, (4, 4), strides=(2, 2), padding='same')(down3)
    down4 = BatchNormalization()(down4)
    down4 = LeakyReLU(alpha=0.2)(down4)

    down5 = Conv2D(512, (4, 4), strides=(2, 2), padding='same')(down4)
    down5 = BatchNormalization()(down5)
    down5 = LeakyReLU(alpha=0.2)(down5)

    down6 = Conv2D(512, (4, 4), strides=(2, 2), padding='same')(down5)
    down6 = BatchNormalization()(down6)
    down6 = LeakyReLU(alpha=0.2)(down6)

    down7 = Conv2D(512, (4, 4), strides=(2, 2), padding='same')(down6)
    down7 = BatchNormalization()(down7)
    down7 = LeakyReLU(alpha=0.2)(down7)

    down8 = Conv2D(512, (4, 4), strides=(2, 2), padding='same')(down7)
    down8 = BatchNormalization()(down8)
    down8 = LeakyReLU(alpha=0.2)(down8)

    # Decoder
    up1 = UpSampling2D(size=(2, 2))(down8)
    up1 = Conv2D(512, (4, 4), padding='same')(up1)
    up1 = BatchNormalization()(up1)
    up1 = Dropout(0.5)(up1)
    up1 = Concatenate()([up1, down7])

    up2 = UpSampling2D(size=(2, 2))(up1)
    up2 = Conv2D(512, (4, 4), padding='same')(up2)
    up2 = BatchNormalization()(up2)
    up2 = Dropout(0.5)(up2)
    up2 = Concatenate()([up2, down6])

    up3 = UpSampling2D(size=(2, 2))(up2)
    up3 = Conv2D(512, (4, 4), padding='same')(up3)
    up3 = BatchNormalization()(up3)
    up3 = Dropout(0.5)(up3)
    up3 = Concatenate()([up3, down5])

    up4 = UpSampling2D(size=(2, 2))(up3)
    up4 = Conv2D(512, (4, 4), padding='same')(up4)
    up4 = BatchNormalization()(up4)
    up4 = Concatenate()([up4, down4])

    up5 = UpSampling2D(size=(2, 2))(up4)
    up5 = Conv2D(256, (4, 4), padding='same')(up5)
    up5 = BatchNormalization()(up5)
    up5 = Concatenate()([up5, down3])

    up6 = UpSampling2D(size=(2, 2))(up5)
    up6 = Conv2D(128, (4, 4), padding='same')(up6)
    up6 = BatchNormalization()(up6)
    up6 = Concatenate()([up6, down2])

    up7 = UpSampling2D(size=(2, 2))(up6)
    up7 = Conv2D(64, (4, 4), padding='same')(up7)
    up7 = BatchNormalization()(up7)
    up7 = Concatenate()([up7, down1])

    # Output layer
    outputs = UpSampling2D(size=(2, 2))(up7)
    outputs = Conv2D(3, (4, 4), padding='same', activation='tanh')(outputs)

    # Define the model
    model = Model(inputs, outputs)
    return model

# Example usage
image_shape = (256, 256, 3)
generator = define_generator(image_shape)
generator.summary()

# Define the Discriminator Model
Define the architecture of the discriminator model using Keras.

In [ ]:
# Define the Discriminator Model
def define_discriminator(image_shape):
    # Input layer for real and generated images
    input_image = Input(shape=image_shape)
    target_image = Input(shape=image_shape)

    # Concatenate the input and target images
    merged = Concatenate()([input_image, target_image])

    # First convolutional layer
    d = Conv2D(64, (4, 4), strides=(2, 2), padding='same')(merged)
    d = LeakyReLU(alpha=0.2)(d)

    # Second convolutional layer
    d = Conv2D(128, (4, 4), strides=(2, 2), padding='same')(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # Third convolutional layer
    d = Conv2D(256, (4, 4), strides=(2, 2), padding='same')(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # Fourth convolutional layer
    d = Conv2D(512, (4, 4), strides=(2, 2), padding='same')(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # Fifth convolutional layer
    d = Conv2D(512, (4, 4), strides=(2, 2), padding='same')(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # Output layer
    d = Conv2D(1, (4, 4), padding='same')(d)
    patch_out = tf.keras.activations.sigmoid(d)

    # Define the model
    model = Model([input_image, target_image], patch_out)
    return model

# Example usage
discriminator = define_discriminator(image_shape)
discriminator.summary()

# Define the Conditional GAN Model
Combine the generator and discriminator models to create the conditional GAN model.

In [ ]:
# Define the Conditional GAN Model
def define_gan(generator, discriminator, image_shape):
    # Make the discriminator not trainable when training the GAN
    discriminator.trainable = False

    # Define the input for the generator
    input_image = Input(shape=image_shape)

    # Generate the target image using the generator
    generated_image = generator(input_image)

    # Get the discriminator's output for the generated image
    gan_output = discriminator([input_image, generated_image])

    # Define the GAN model
    model = Model(input_image, [gan_output, generated_image])

    # Compile the model
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

    return model

# Example usage
gan = define_gan(generator, discriminator, image_shape)
gan.summary()

# Compile the Models
Compile the generator, discriminator, and conditional GAN models with appropriate loss functions and optimizers.

In [6]:
# Compile the Models

# Compile the generator model
generator.compile(loss='mae', optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Compile the discriminator model
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])

# Compile the conditional GAN model
gan.compile(loss=['binary_crossentropy', 'mae'], optimizer=Adam(learning_rate=0.0002, beta_1=0.5))

# Train the Conditional GAN
Train the conditional GAN model on the preprocessed dataset, including defining the training loop and updating the models.

In [ ]:
# Train the Conditional GAN

# Define the training function
def train_gan(generator, discriminator, gan, dataset, epochs=100, batch_size=1):
    # Calculate the number of batches per epoch
    batch_count = dataset.shape[0] // batch_size

    for epoch in range(epochs):
        for batch in range(batch_count):
            # Select a random batch of images
            idx = np.random.randint(0, dataset.shape[0], batch_size)
            real_images = dataset[idx]

            # Generate fake images
            fake_images = generator.predict(real_images)

            # Create labels for real and fake images
            real_labels = np.ones((batch_size,) + discriminator.output_shape[1:])
            fake_labels = np.zeros((batch_size,) + discriminator.output_shape[1:])

            # Train the discriminator
            d_loss_real = discriminator.train_on_batch([real_images, real_images], real_labels)
            d_loss_fake = discriminator.train_on_batch([real_images, fake_images], fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train the generator (via the GAN model)
            g_loss = gan.train_on_batch(real_images, [real_labels, real_images])

            # Print the progress
            print(f'Epoch: {epoch+1}/{epochs}, Batch: {batch+1}/{batch_count}, D Loss: {d_loss[0]}, D Acc: {d_loss[1]}, G Loss: {g_loss[0]}')

# Example usage
train_gan(generator, discriminator, gan, train_images, epochs=100, batch_size=1)

# Generate and Visualize Translated Images
Use the trained generator model to generate translated images and visualize the results.

In [ ]:
# Generate and Visualize Translated Images

# Define function to denormalize images from range [-1, 1] to [0, 1]
def denormalize_image(image):
    return (image + 1.0) / 2.0

val_images = load_dataset('data/mini_pix2pix/val')

# Select a random sample of validation images
num_samples = 5
sample_indices = np.random.choice(val_images.shape[0], num_samples, replace=False)
sample_images = val_images[sample_indices]

# Generate translated images using the trained generator
translated_images = generator.predict(sample_images)

# Plot the original and translated images
plt.figure(figsize=(15, 10))
for i in range(num_samples):
    # Original image
    plt.subplot(2, num_samples, i + 1)
    plt.imshow(denormalize_image(sample_images[i]))
    plt.axis('off')
    plt.title('Original')

    # Translated image
    plt.subplot(2, num_samples, num_samples + i + 1)
    plt.imshow(denormalize_image(translated_images[i]))
    plt.axis('off')
    plt.title('Translated')

plt.show()